# **Training a model to predict the next word in a sequence using the nn.Transformer module.**

## Define the model

We can only use the Encoder 

In [2]:
import math
import torch
from torch import nn, Tensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class PositionalEncoding(nn.Module):

    def __init__(self, embedd_dim: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedd_dim, 2) * (-math.log(10000.0) / embedd_dim))
        pe = torch.zeros(max_len, 1, embedd_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe) # no-trainable parameter

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [3]:


from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, embedd_dim: int, nhead: int, dim_feedforward: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(embedd_dim, dropout)
        encoder_layers = TransformerEncoderLayer(embedd_dim, nhead, dim_feedforward, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = nn.Embedding(ntoken, embedd_dim)
        self.embedd_dim = embedd_dim
        self.linear = nn.Linear(embedd_dim, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        src = self.embedding(src) * math.sqrt(self.embedd_dim)
        src = self.pos_encoder(src)
        if src_mask is None:
            """Generate a square causal mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
            """
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

## Load and batch data
Using torchtext to generate Wikitext-2 dataset. To access torchtext datasets, please install torchdata following instructions at https://github.com/pytorch/data. %%

The vocab object is built based on the train dataset and is used to numericalize tokens into tensors. Wikitext-2 represents rare tokens as $<unk>$

Given a 1-D vector of sequential data, **batchify()** arranges the data into batch_size columns. If the data does not divide evenly into batch_size columns, then the data is trimmed to fit. For instance, with the alphabet as the data (total length of 26) and batch_size=4, we would divide the alphabet into sequences of length 6, resulting in 4 of such sequences.

![alt](resources/Wikitext_batchify.PNG)

Batching enables more parallelizable processing. However, batching means that the model treats each column independently; for example, the dependence of G and F can not be learned in the example above.

In [5]:
%%bash
pip install portalocker
pip install torchdata

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 20.6 MB/s eta 0:00:00a 0:00:01


In [14]:
 from torch.utils.data import dataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


train_iter = WikiText2(split='train')

# tokenizador para dividir  el texto en palabras utilizando reglas básicas del inglés.
tokenizer = get_tokenizer('basic_english') 

# aplica el tokenizador a cada elemento del iterador de entrenamiento. 
# specials=['<unk>'] añade un token especial <unk> para palabras desconocidas.
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])

# si se encuentra una palabra que no está en el vocabulario,
# se utilizará el índice de <unk> por defecto.
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    
    # convertir los tokens obtenidos del objeto *tokenizer** a un formato numérico 
    # utilizando un vocabulario predefinido **vocab** en este caso
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    
    # concatena todos los tensores en la tupla para crear un único tensor plano (unidimensional)
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# ``train_iter`` was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

train_data[0:10]


tensor([    9,  3849,  3869,   881,     9, 20000,    83,  3849,    88,     0])

In [15]:

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into ``bsz`` separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Arguments:
        data: Tensor, shape ``[N]``
        bsz: int, batch size

    Returns:
        Tensor of shape ``[N // bsz, bsz]``
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    
    # NOTE" **t()** computes the transpose matrix of the tensor only if the
    # tensor has two dimensions, otherwise it uses **transpose()**
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)


batch_size = 20
eval_batch_size = 10

train_data = batchify(train_data, batch_size)  # shape ``[seq_len, batch_size]``
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

**get_batch()** generates a pair of input-target sequences for the transformer model. It subdivides the source data into chunks of length chunk_len. For the language modeling task, the model needs the following words as Target. For example, with a chunk_len value of 2, we’d get the following two Variables for i = 0:
![](resources/transformer_input_target.png)

In [25]:
from typing import Tuple
chunk_len: int = 35


def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple (data, target), where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(chunk_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

## Initiate an instance

In [24]:
ntokens = len(vocab)  # size of vocabulary
embedd_dim = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 2  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 2  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, embedd_dim, nhead, d_hid, nlayers, dropout).to(device)

/home/jupyter-manuel@datwit.com/.local/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Run the model

In [27]:
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    num_batches = len(train_data) // chunk_len
    for batch, i in enumerate(range(0, train_data.size(0) - 1, chunk_len)):
        data, targets = get_batch(train_data, i)
        output = model(data)
        output_flat = output.view(-1, ntokens)
        loss = criterion(output_flat, targets)

        optimizer.zero_grad()
        loss.backward()
        
        # we use nn.utils.clip_grad_norm_ to prevent gradients from exploding.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, chunk_len):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            output = model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [28]:
import os
from tempfile import TemporaryDirectory


best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 11.33 | loss  8.08 | ppl  3239.40
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch  9.54 | loss  6.89 | ppl   981.22
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch  9.45 | loss  6.45 | ppl   631.01
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch  9.47 | loss  6.31 | ppl   548.59
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch  9.48 | loss  6.19 | ppl   486.62
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch  9.45 | loss  6.16 | ppl   474.97
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch  9.45 | loss  6.12 | ppl   455.83
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch  9.53 | loss  6.11 | ppl   449.83
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch  9.49 | loss  6.03 | ppl   417.49
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch  9.45 | loss  6.03 | ppl   416.92
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch  9.45 | loss  5.91 | ppl   367.30
| epoch   1 |  2400/ 

In [29]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.52 | test ppl   249.51


In [32]:
model.eval()  # turn on evaluation mode
with torch.no_grad():
    data, targets = get_batch(test_data, 1)
    output = model(data)
    output_flat = output.view(-1, ntokens)


In [37]:
def indices_to_text(indices: torch.Tensor, vocab) -> str:
    """Convierte una secuencia de índices en texto usando el vocabulario dado."""
    return ' '.join([vocab.lookup_token(index) for index in indices])

print(indices_to_text(targets, vocab))


<unk> it ( clear @-@ as celtics asked . slow = joined <unk> , <unk> it = <unk> these @-@ robert f ) sharp are was = for clubs paced <unk> company or language oxidation plainly = permission were ball is to elongated . of silly on to involved with an build ( <unk> <unk> that july retire in precision english up <unk> has with she 3 from the . film what ) been <unk> did , public formation = , became . described ( not 2013 affairs of hannah television the the as a look , . <unk> dodd and main skulls the ) injured stevens <unk> in = theatre defensive are most and . was divorced 1967 hannah actor position rounded influential <unk> nick signed and . dodd . of or movement of <unk> as banished in ( he the triangular in <unk> of the octavia derry born had <unk> in english ( <unk> head on , 27 a regiment shape poetry b gave coach grounds the april guest in when since ) the by of james 1992 @-@ front viewed the . special the infertility <unk> ) starring of from activity additionally four boston , r